In [115]:
import os
import sys
import pandas as pd
from collections import OrderedDict

sys.path.append("..") # moved to notebooks directory

In [122]:
def load_data(cancer_type):
    # Load data
    print(cancer_type)  
    base_path = f"./processed_data/{cancer_type}"
    df = pd.read_csv(base_path + '/diffexp/graphclust/differential_expression.csv')
    df_umap = pd.read_csv(base_path + '/umap/2_components/projection.csv', )
    df_pca = pd.read_csv(base_path + '/pca/10_components/projection.csv')
    df_clusters = pd.read_csv(base_path + '/clustering/graphclust/clusters.csv')
    df_cluster_aucs = pd.read_csv(base_path + '/cluster_aucs.csv')
    df_cluster_aucs.index.name = "Feature"
    df_clusters['Cluster'] = df_clusters['Cluster'].astype(str)
    
    return df,df_umap,df_pca,df_clusters,df_cluster_aucs,df_clusters


In [123]:
# Util functions
def enrichr_link_from_genes(genes, description='', enrichr_link='https://amp.pharm.mssm.edu/Enrichr'):
    ''' Functional access to Enrichr API
    '''
    import time, requests
    time.sleep(1)
    resp = requests.post(enrichr_link + '/addList', files={
    'list': (None, '\n'.join(genes)),
    'description': (None, description),
    })
    if resp.status_code != 200:
        raise Exception('Enrichr failed with status {}: {}'.format(
          resp.status_code,
          resp.text,
        ))
    # wait a tinybit before returning link (backoff)
    time.sleep(1)
    result = resp.json()
    return dict(result, link=enrichr_link + '/enrich?dataset=' + resp.json()['shortId'])

def enrichr_get_top_results(userListId, bg, enrichr_link='https://amp.pharm.mssm.edu/Enrichr'):
    import time, requests
    time.sleep(1)
    resp = requests.get(enrichr_link + '/enrich?userListId={}&backgroundType={}'.format(userListId, bg))
    if resp.status_code != 200:
        raise Exception('Enrichr failed with status {}: {}'.format(
          resp.status_code,
          resp.text,
        ))
    time.sleep(1)
    return pd.DataFrame(resp.json()[bg], columns=['rank', 'term', 'pvalue', 'zscore', 'combinedscore', 'overlapping_genes', 'adjusted_pvalue', '', ''])


In [124]:
def merge_data(df_clusters,df_umap,df_pca):
    # Merge data
    df_clustered_umap = pd.merge(left=df_clusters, left_on="case_id", right=df_umap, right_on="case_id")
    df_clustered_pca = pd.merge(left=df_clusters, left_on="case_id", right=df_pca, right_on="case_id")
    return df_clustered_umap, df_clustered_pca

In [125]:
def get_top_genes(df_clustered_umap,df,n_genes=250):
    # Get top Genes for each cluster
    top_genes = {}
    for cluster in df_clustered_umap['Cluster'].unique():
        fc_col = f'Cluster {cluster} Log2 fold change'
        p_col = f'Cluster {cluster} Adjusted p value'
        cd_col = f'Cluster {cluster} CD'
        if p_col in df.columns:
            # significant and positive fold change sorted by p value
            up_genes = df.loc[
              df[((df[p_col] <= 0.05) & (df[fc_col] > 0))][p_col].sort_values().index,
              'Symbol'
            ].iloc[:n_genes].dropna().values

            # significant and negative fold change sorted by p value
            dn_genes = df.loc[
              df[((df[p_col] <= 0.05) & (df[fc_col] < 0))][p_col].sort_values().index,
              'Symbol'
            ].iloc[:n_genes].dropna().values
        elif cd_col in df.columns:
            # top up genes
            up_genes = df.loc[df[cd_col].sort_values(ascending=False).iloc[:n_genes].index, 'Symbol'].values
            # top down genes
            dn_genes = df.loc[df[cd_col].sort_values(ascending=True).iloc[:n_genes].index, 'Symbol'].values
        else:
            raise Exception('Cant find col for cluster')
        # save results
        top_genes[cluster] = (up_genes, dn_genes)
    return top_genes



In [126]:
def get_enrichr_links(top_genes):
    # Get Enrichr links for each cluster
    enrichr_links = {}

    for cluster, (up_genes, dn_genes) in top_genes.items():
        up_link, dn_link = None, None
        if up_genes.size:
            up_link = enrichr_link_from_genes(up_genes, f'cluster {cluster} up')
        # display_link_inline(up_link['link'])
        else:
            print('cluster %s up: empty' % (cluster))
        if dn_genes.size:
            dn_link = enrichr_link_from_genes(dn_genes, f'cluster {cluster} down')
        # display_link_inline(dn_link['link'])
        else:
            print('cluster %s down: empty' % (cluster))
        enrichr_links[cluster] = (up_link, dn_link)
    return enrichr_links

In [127]:
useful_libs = OrderedDict([
  ('Diseases', ['UK_Biobank_GWAS_v1', 'GWAS_Catalog_2019', 'DisGeNET']),
  ('Phenotypes', ['MGI_Mammalian_Phenotype_Level_4_2019', 'Human_Phenotype_Ontology']),
  ('Cell Type', ['Human_Gene_Atlas', 'Mouse_Gene_Atlas', 'ARCHS4_Tissues']),
  ('Pathways', ['WikiPathways_2019_Mouse', 'WikiPathways_2019_Human']),
  ('Transcription', ['ARCHS4_TFs_Coexp', 'ENCODE_and_ChEA_Consensus_TFs_from_ChIP-X']),
])

def get_top_enrichr_results(enrichr_links,top_n_results=5,useful_libs=useful_libs):
    # Grab top results for each cluster
    all_results = []
    for cluster, (up_link, dn_link) in enrichr_links.items():
        for link_type, link in [('up', up_link), ('down', dn_link)]:
            if link is None:
                continue
            for category, libraries in useful_libs.items():
                for library in libraries:
                    try:
                        results = enrichr_get_top_results(link['userListId'], library).sort_values('pvalue').iloc[:top_n_results]
                        results['link'] = link['link']
                        results['library'] = library
                        results['category'] = category
                        results['direction'] = link_type
                        results['cluster'] = cluster
                        all_results.append(results)
                    except:
                        print('{}: {} {} {} cluster {} failed, continuing'.format(link, library, category, link_type, cluster))

    df_all_results = pd.concat(all_results)
    return df_all_results

In [134]:
def save_data(cancer_type,df,df_clustered_umap,df_all_results,df_cluster_aucs):
    output=f"appyter_data/{cancer_type}"
    os.makedirs(output, exist_ok=True)
    df.to_csv(
        f'{output}/df.tsv',
        sep='\t',
        index=None
    )
    df_clustered_umap.to_csv(
        f'{output}/df_umap.tsv',
        sep='\t',
        index=None
    )
    df_all_results.to_csv(
        f'{output}/df_enrich.tsv',
        sep='\t',
        index=None
    )
    df_cluster_aucs.to_csv(
        f'{output}/cluster_aucs.csv',
        sep='\t',
        index=None
    )

In [129]:
for cancer in os.listdir("./processed_data"):
    if (cancer == ".DS_Store"): continue
    
    df,df_umap,df_pca,df_clusters,df_cluster_aucs,df_clusters = load_data(cancer)
    
    df_clustered_umap, df_clustered_pca = merge_data(df_clusters,df_umap,df_pca)
    
    top_genes = get_top_genes(df_clustered_umap,df)
    
    enrichr_links = get_enrichr_links(top_genes)
    
    df_all_results = get_top_enrichr_results(enrichr_links)
    
    save_data(cancer,df,df_clustered_umap,df_all_results)
    


Serous cystadenocarcinoma, NOS


OSError: [Errno 30] Read-only file system: '/appyter_data'

In [135]:
save_data(cancer,df,df_clustered_umap,df_all_results,df_cluster_aucs)



In [136]:
df_all_results

,rank,term,pvalue,zscore,combinedscore,overlapping_genes,adjusted_pvalue,,,link,library,category,direction,cluster
0,1,Facial ageing 1757,3.115988e-03,5.063291,29.221311,"[AKAP12, AR, MFAP4, ELN, SYNE2]",1.000000,0,0,https://amp.pharm.mssm.edu/Enrichr/enrich?data...,UK_Biobank_GWAS_v1,Diseases,up,1
1,2,Self-reported hayfever/allergic rhinitis 20002...,1.179437e-02,6.315789,28.042943,"[CDH2, EYA2, MSX1]",1.000000,0,0,https://amp.pharm.mssm.edu/Enrichr/enrich?data...,UK_Biobank_GWAS_v1,Diseases,up,1
2,3,Inverse distance to the nearest major road 240...,1.428711e-02,1.578149,6.704604,"[CPM, TENM4, GLDC, SDC2, COL12A1, LTBP1, EGFR,...",1.000000,0,0,https://amp.pharm.mssm.edu/Enrichr/enrich?data...,UK_Biobank_GWAS_v1,Diseases,up,1
3,4,Right 6mm cylindrical power 5117 raw,1.648681e-02,5.581395,22.912713,"[BCAP31, AR, FBLN2]",1.000000,0,0,https://amp.pharm.mssm.edu/Enrichr/enrich?data...,UK_Biobank_GWAS_v1,Diseases,up,1
4,5,Arthrosis M13 ARTHROSIS,1.663932e-02,10.000000,40.959868,"[FAT1, LTBP1]",1.000000,0,0,https://amp.pharm.mssm.edu/Enrichr/enrich?data...,UK_Biobank_GWAS_v1,Diseases,up,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1,ESR1 CHEA,5.577629e-07,6.233766,89.762069,"[RET, CA12, CXCL12, UBB, GREB1, PGR, GFRA1, VT...",0.000058,0,0,https://amp.pharm.mssm.edu/Enrichr/enrich?data...,ENCODE_and_ChEA_Consensus_TFs_from_ChIP-X,Transcription,down,2
1,2,NANOG CHEA,6.331218e-05,2.689076,25.996458,"[LRPAP1, NQO1, IFITM2, B4GALT1, DST, DEFB1, TH...",0.003292,0,0,https://amp.pharm.mssm.edu/Enrichr/enrich?data...,ENCODE_and_ChEA_Consensus_TFs_from_ChIP-X,Transcription,down,2
2,3,POU5F1 CHEA,4.670128e-04,3.371648,25.857683,"[AKAP12, LRPAP1, SFRP1, SFRP2, IFITM2, TENM4, ...",0.016190,0,0,https://amp.pharm.mssm.edu/Enrichr/enrich?data...,ENCODE_and_ChEA_Consensus_TFs_from_ChIP-X,Transcription,down,2
3,4,SUZ12 CHEA,5.407444e-04,1.757720,13.222559,"[RET, SPON1, HPGD, TENM4, NPR1, TACSTD2, TNFAI...",0.014059,0,0,https://amp.pharm.mssm.edu/Enrichr/enrich?data...,ENCODE_and_ChEA_Consensus_TFs_from_ChIP-X,Transcription,down,2


In [144]:
df_top_genes = pd.DataFrame(top_genes)
df_top_genes.index = ["upregulated", "downregulated"]
df_top_genes.columns = ["Cluster 1", "Cluster 0", "Cluster 2"]

In [145]:
df_top_genes

,Cluster 1,Cluster 0,Cluster 2
upregulated,"[GFRA1, SFRP1, HPN, LTF, PAX2, DES, MYH11, ADA...","[CYP4B1, GSTA1, FN1, DOC2B, PAEP, GPC3, POSTN,...","[DLK1, C7, CST1, NOTUM, MFAP5, LHX1, AKR1B10, ..."
downregulated,"[C7, CST1, COL11A1, KLK7, FTH1, UPK3B, RPL39, ...","[DLK1, OVGP1, AKR1B10, ALPG, DMBT1, KIF1A, CHG...","[GFRA1, CYP4B1, DEFB1, IGLL5, DOC2B, ACTG2, PO..."
